In [0]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Energy Forecasting with AutoML Tables**

<table align="left">
  <td>
    <a href="https://cloud.google.com/ml-engine/docs/tensorflow/getting-started-keras">
      <img src="https://cloud.google.com/_static/images/cloud/icons/favicons/onecloud/super_cloud.png"
           alt="Google Cloud logo" width="32px"> Read on cloud.google.com
    </a>
  </td>
  <td>
    <a href="#">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="#">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## **Overview**


This guide provides a high-level overview of an energy price forecasting solution, considering the significance of the solution as well as which audiences and use cases it applies to. In this section, we outline the business case for this solution, the problem, the solution, and results. In section 2, we provide the code setup instructions.

**Solution description:** Model to forecast hourly energy prices for the next 7 days.

**Significance:** This is a good complement to standard demand forecasting models that typically predict N periods in the future. This model does a rolling forecast that is vital for operational decisions. It also takes into consideration historical trends, seasonal patterns, and external factors (like weather) to make more accurate forecasts.

### **Dataset**



#### **1. Download raw data**

  Download the raw data for the model from the links below.
* [MarketPricePT](http://complatt.smartwatt.net/assets/files/historicalRealData/RealMarketPriceDataPT.csv) - Historical hourly energy prices.

* [historical_weather](http://complatt.smartwatt.net/assets/files/weatherHistoricalData/WeatherHistoricalData.zip) - Historical hourly weather forecasts.

*Disclaimer: The data for both tables comes from http://complatt.smartwatt.net/. This website hosts a closed competition meant to solve the energy price forecasting problem. The data was not collected or vetted by Google LLC and hence, we cannot guarantee the veracity or quality of it.

#### **2. Create BigQuery table**


1. Create dataset named `Energy`

2. Create table named "MarketPricePT" using downloaded `RealMarketPriceDataPT.csv` file. Make sure the column names are date_utc, price and delimitor as comma(,) in the file

3. Create table named "historical_weather" using downloaded `historical_weather.csv` file







#### **3. Execute script for data preparation**
Prepare the data that is going to be used by the forecaster model by following below instructions:

1. Clone the solution code from here: https://github.com/GoogleCloudPlatform/professional-services/tree/master/examples/cloudml-energy-price-forecasting. Navigate to the `"data_preparation"` folder.

2. Run script `"data_preparation.data_prep"` to generate training, validation, and testing data as well as the constant files needed for normalization. This will create three BigQuery tables `MLDataTest`, `MLDataTrain` & `MLDataValid`.

3. Read the ["README.md"](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/tables/automl/notebooks/energy_price_forecasting/README.md) file for more information.

4. Understand which parameters can be passed to the script (to override defaults).

### **Costs**


This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage
* BigQuery
* AutoML Tables

Learn about [Cloud AI Platform pricing](https://cloud.google.com/ml-engine/docs/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing),[Bigquery pricing](https://cloud.google.com/bigquery/pricing), [AutoML Tables pricing](https://cloud.google.com/automl-tables/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## **Set up your local development environment**

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. If you are using **AI Platform Notebook**, make sure the machine configuration type is **1 vCPU, 3.75 GB RAM** or above. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

2. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

3. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3.

4. Activate that environment and run `pip install jupyter` in a shell to install
   Jupyter.

5. Run `jupyter notebook` in a shell to launch Jupyter.

6. Open this notebook in the Jupyter Notebook Dashboard.

## **Set up your GCP project**

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Enable AutoML API.](https://console.cloud.google.com/apis/library/automl.googleapis.com?q=automl)


## **PIP Install Packages and dependencies**

Install addional dependencies not installed in the notebook environment.

In [0]:
# Use the latest major GA version of the framework.
! pip install --quiet google-cloud-automl 
! pip install --quiet google-cloud-bigquery

**Note:** Try installing using `sudo`, if the above command throw any permission errors.

`Restart` the kernel to allow `automl_v1beta1` to be imported for Jupyter Notebooks.

In [0]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## **Set up your GCP Project Id**

Enter your `Project Id` in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

In [0]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}
COMPUTE_REGION = "us-central1" # Currently only supported region.

## **Authenticate your GCP account**

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

Otherwise, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **AutoML > AutoML Admin**, 
   **Storage > Storage Object Admin**, 
   **BigQuery > BigQuery Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [0]:
import sys

# Upload the downloaded JSON file that contains your key.
if 'google.colab' in sys.modules:    
  from google.colab import files
  keyfile_upload = files.upload()
  keyfile = list(keyfile_upload.keys())[0]
  %env GOOGLE_APPLICATION_CREDENTIALS $keyfile
  ! gcloud auth activate-service-account --key-file $keyfile

***If you are running the notebook locally***, enter the path to your service account key as the `GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell

In [0]:
# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.

%env GOOGLE_APPLICATION_CREDENTIALS /path/to/service/account
! gcloud auth activate-service-account --key-file '/path/to/service/account'

## **Import libraries and define constants**

Import relevant packages.


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [0]:
from google.cloud import automl_v1beta1 as automl
from google.cloud import bigquery
import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types

Populate the following cell with the necessary constants and run it to initialize constants.

In [0]:
#@title Constants { vertical-output: true }

# A name for the AutoML tables Dataset to create.
DATASET_DISPLAY_NAME = 'energy_forcasting_solution' #@param {type:'string'}
# The BigQuery dataset to import data from.
BQ_DATASET_NAME = "Energy" 
# The BigQuery table to import data from.
BQ_TABLE_NAME = "automldata"
# A name for the AutoML tables model to create.
MODEL_DISPLAY_NAME = 'energy_model' #@param {type:'string'}

assert all([
    PROJECT_ID,
    COMPUTE_REGION,
    DATASET_DISPLAY_NAME,
    BQ_DATASET_NAME,
    BQ_TABLE_NAME,
    MODEL_DISPLAY_NAME,
])

Initialize the clients for AutoML, AutoML Tables and BigQuery.

In [0]:
# Initialize the clients.
automl_client = automl.AutoMlClient()
tables_client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION)
bq_client = bigquery.Client()

## **Test the set up**

To test whether your project set up and authentication steps were successful, run the following cell to list your datasets in this project.

If no dataset has previously imported into AutoML Tables, you shall expect an empty return.

In [0]:
# List the datasets.
list_datasets = tables_client.list_datasets()
datasets = { dataset.display_name: dataset.name for dataset in list_datasets }
datasets

You can also print the list of your models by running the following cell.

If no model has previously trained using AutoML Tables, you shall expect an empty return.


In [0]:
# List the models.
list_models = tables_client.list_models()
models = { model.display_name: model.name for model in list_models }
models

## **Import training data**

### **Create dataset**
Select a dataset display name and pass your table source information to create a new dataset.

In [0]:
# Create dataset.
dataset = tables_client.create_dataset(
    dataset_display_name=DATASET_DISPLAY_NAME)
dataset_name = dataset.name
dataset

### **Concat Train, Test & Valid data**
Following query will concatenate Train, Valid and Test data in to a single table. Column 'split' represents the train/valid/test data. Modify the dataset and table name (If necessary).

In [0]:
query = '''CREATE TABLE `{0}.{1}.{2}` AS  (
  SELECT * FROM (
    SELECT *,"TEST" as split FROM `{0}.{1}.MLDataTest`  
    union all 
    SELECT *,"TRAIN" as split FROM `{0}.{1}.MLDataTrain` 
    union all 
    SELECT *,"VALIDATE" as split FROM `{0}.{1}.MLDataValid`
    ))'''.format(PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME)
bq_client.query(query,)

### **Import data**


You can import your data to AutoML Tables from GCS or BigQuery. You can create a GCS bucket and upload the data into your bucket. The URI for your file is `gs://BUCKET_NAME/FOLDER_NAME1/FOLDER_NAME2/.../FILE_NAME`. Alternatively you can create a BigQuery table and upload the data into the table. The URI for your table is `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

Importing data may take a few minutes or hours depending on the size of your data. **If your Colab times out**, run the following command to retrieve your dataset. Replace `dataset_name` with its actual value obtained in the preceding cells.

    # This will work if your display name ('energy_forecasting_solution') is unique to your project.
      dataset = tables_client.get_dataset(dataset_display_name=DATASET_DISPLAY_NAME)
    # OR, if you have multiple datasets with the same display name ('energy_forecasting_solution'), use the
    # unique indentifier acquired from the above cell ( print(dataset.name) ).
      dataset = tables_client.get_dataset(dataset_name=dataset_name)

In [0]:
# Import data from BigQuery.
dataset_bq_input_uri = "bq://{}.{}.{}".format(
    PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME)

import_data_response = tables_client.import_data(
    dataset=dataset,
    bigquery_input_uri=dataset_bq_input_uri
)

print('Dataset import operation: {}'.format(import_data_response.operation))

# Wait until import is done.
print('Dataset import result: {}'.format(import_data_response.result()))

# Verify the status by checking the example_count field.
dataset = tables_client.get_dataset(dataset_name=dataset_name)
dataset

## **Review the specs**
Run the following command to see table specs such as row count.

In [0]:
# List table specs.
list_table_specs_response = tables_client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs.
list_column_specs_response = tables_client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type.
features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) 
            for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

## **Update dataset: assign a label column and enable nullable columns**


AutoML Tables automatically detects your data column type. For example, for the [Iris dataset](https://storage.cloud.google.com/rostam-193618-tutorial/automl-tables-v1beta1/iris.csv) it detects `species` to be categorical and `petal_length`, `petal_width`, `sepal_length`, and `sepal_width` to be numerical. Depending on the type of your label column, AutoML Tables chooses to run a classification or regression model. If your label column contains only numerical values, but they represent categories, change your label column type to categorical by updating your schema.

### **Update a column: set as categorical**

In [0]:
# Update column.
column_to_category = 'hour' #@param {type: 'string'}
type_code = 'CATEGORY' #@param {type: 'string'}
update_column_response = tables_client.update_column_spec(
    dataset=dataset,
    column_spec_display_name=column_to_category,
    type_code=type_code
)

update_column_response.display_name, update_column_response.data_type

### **Update dataset: assign a target and split column**

In [0]:
# Set target and split column.
target_column_name = 'price' #@param {type: 'string'}
split_column_name = 'split' #@param {type: 'string'}

tables_client.set_target_column(
    dataset=dataset,
    column_spec_display_name=target_column_name,
)

tables_client.set_test_train_column(
    dataset=dataset,
    column_spec_display_name=split_column_name,
)

## **Creating a model**

### **Train a model**
Specify the duration of the training. For example, `'train_budget_milli_node_hours': 1000` runs the training for one hour. You can increase that number up to a maximum of 72 hours `('train_budget_milli_node_hours': 72000)` for the best model performance.

Even with a budget of 1 node hour (the minimum possible budget), training a model can take more than the specified node hours

If your Colab times out, use `tables_client.list_models()` to check whether your model has been created. Then use model name to continue to the next steps. Run the following command to retrieve your model.

    model = tables_client.get_model(model_display_name=MODEL_DISPLAY_NAME)

You can also select the objective to optimize your model training by setting optimization_objective. This solution optimizes the model by minimizing mean absolute error (MAE).

In [0]:
# The number of hours to train the model.
model_train_hours = 1 #@param {type:'integer'}
# Set optimization objective to train the model.
model_optimization_objective = 'MINIMIZE_MAE' #@param {type:'string'}

create_model_response = tables_client.create_model(
    model_display_name=MODEL_DISPLAY_NAME,
    dataset=dataset,
    optimization_objective=model_optimization_objective,
    train_budget_milli_node_hours=model_train_hours*1000,
)

operation_id = create_model_response.operation.name

print('Create model operation: {}'.format(create_model_response.operation))

In [0]:
# Wait until model training is done.
model = create_model_response.result()
model_name = model.name
model

## **Evaluation Metrics**

In [0]:
# List evaluation metrics.
metrics = [x for x in tables_client.list_model_evaluations(model=model)][-1]
metrics.regression_evaluation_metrics

## **Get Model**

In [0]:
# Get model info.
model = tables_client.get_model(model_name=model_name)
model.name

## **Feature Importance**

In [0]:
# List column features.
feat_list = [(x.feature_importance, x.column_display_name) 
            for x in model.tables_model_metadata.\
            tables_model_column_info]
feat_list.sort(reverse=True)
# Only the top features are listed.
feat_list[:15]

## **Cleaning up**

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

**Delete BigQuery datasets**

In order to delete BigQuery tables, make sure the service account linked to this notebook has a role with the bigquery.tables.delete permission such as Big Query Data Owner. The following command displays the current service account.

IAM permissions can be adjusted [here](https://console.cloud.google.com/navigation-error;errorUrl=%2Fiam-admin%2Fiam%3Fproject%3Dprj-automl-notebook&folder%3D&organizationId%3D/permissions).

In [0]:
# Delete model resource.
tables_client.delete_model(model_name=model_name)

# Delete dataset resource.
tables_client.delete_dataset(dataset_name=dataset_name)

# Delete the training dataset.
dataset_id = "{0}.{1}".format(PROJECT_ID, BQ_DATASET_NAME)
bq_client.delete_dataset(dataset_id, delete_contents=True, not_found_ok=True)

# If training model is still running, cancel it.
automl_client.transport._operations_client.cancel_operation(operation_id)